In [ ]:
# -*- coding: utf-8 -*-
# python3 example_cifar10_scnn.py
import sys
sys.path.append("..")
from importlib import import_module
import torch,time,os,random
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append("..")
import LIAF
from datasets.cifar10_dvs_simple import cifar10_DVS
from tensorboardX import SummaryWriter
from importlib import import_module
from LIAFnet.LIAFResNet import *
autocast = LIAF.autocast #统一autocast模式

################################ parameters ####################

for test in range(0,10):
    os.environ['CUDA_VISIBLE_DEVICES'] = "1"
    task = 'LongLongwithout_pretarin' + str(test)
    writer = SummaryWriter(comment=task)
    learning_rate = 1e-3
    batch_size  = 256
    num_epochs = 100
    timeWindows = 8
    names = 'dvs_cifar10'
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_path = '/data/CIFAR10-DVS-mini/train.mat' #r := raw string
    test_path =  '/data/CIFAR10-DVS-mini/test.mat' #TODO:input your oath

    train_dataset = cifar10_DVS(train_path,'r',timeWindows) #max= 25?
    test_dataset = cifar10_DVS(test_path,'r',timeWindows)

    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                               batch_size=batch_size,
                                               shuffle=False)

    TA.timeWindows = 8 #用了attention必须固定时间长度
    modules = import_module('LIAFnet.LIAFResNet_18')
    config  = modules.Config()
    workpath = os.path.abspath(os.getcwd())
    accumulation = config.accumulation
    timeWindows = config.timeWindows
    config.cfgCnn = [2, 64, 7, True]
    config.dataSize = (42,42)
    config.actFun= LIAF.LIFactFun.apply
    epoch = 0
    bestacc = 0  # best test accuracy
    start_epoch = 0  # start from epoch 0 or last checkpoint epoch
    training_iter = 0
    snn = LIAFResNet(config)
    snn.fc = nn.Linear(snn.cfgFc_[0],10)
    print("Total number of paramerters in networks is {}  ".format(sum(x.numel() for x in snn.parameters())))

    snn.to(device)
    criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(snn.parameters(),
                    lr=learning_rate,
                    weight_decay =1e-4)


    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(
                        optimizer, 
                        milestones=[90,180], 
                        gamma=0.1, 
                        last_epoch=-1)
    def val(optimizer,snn,test_loader,test_dataset,batch_size,epoch):
        print('===> evaluating models...')
        snn.eval()
        correct = 0
        total = 0
        predicted = 0
        with torch.no_grad():
            for batch_idx, (inputs, labels) in enumerate(test_loader):
                inputs = inputs.permute([0,1,4,2,3])
                if ((batch_idx+1)<=len(test_dataset)//batch_size):
                    optimizer.zero_grad()
                    outputs = snn(inputs.to(device))

                    _ , labels = labels.max(1)
                    _ , predicted = outputs.cpu().max(1)
                    total += float(labels.size(0))
                    correct += float(predicted.eq(labels).sum())
        acc = 100. * float(correct) / float(total)
        print('================')
        print('val acc:',acc , '%  epoch:',epoch)
        print('================')
        writer.add_scalar('acc', acc, epoch)
        return acc

    save_folder = './saved_model'

    for epoch in range(num_epochs):
        # 新增2：设置sampler的epoch，DistributedSampler需要这个来维持各个进程之间的相同随机数种子
        #train_loader.sampler.set_epoch(epoch)
        snn.train()
        running_loss = 0
        start_time = time.time() 
        print('===> training models...')
        correct = 0.0
        total = 0.0
        torch.cuda.empty_cache()
        for i, (images, labels) in enumerate(train_loader):
            images = images.permute([0,1,4,2,3])
            if ((i+1)<=len(train_dataset)//batch_size):
                with autocast():
                    outputs = snn(images.to(device)).cpu()
                    _,labels = labels.max(1)
                    _ , predict = outputs.max(1)
                    loss = criterion(outputs, labels)
                    correct += predict.eq(labels).sum()
                    total += float(predict.size(0))

                    loss /= accumulation
                    running_loss += loss.item()
                    loss.backward()

                if (i+1)%accumulation == 0:
                    optimizer.step()
                    snn.zero_grad()
                    optimizer.zero_grad()

                if (i+1)%(accumulation*18) == 0:
                    if not os.path.isdir(save_folder):
                        os.mkdir(save_folder)
                    train_acc =  100* correct / total
                    print('=====> Epoch [%d/%d], Step [%d/%d], Loss: %.5f , Acc: %.5f  '
                      %(epoch+start_epoch, num_epochs+start_epoch, i+1, len(train_dataset)//(batch_size),running_loss, train_acc))
                    writer.add_scalar('running Loss', running_loss, training_iter)
                    writer.add_scalar('running Acc', train_acc, training_iter)
                    correct = 0.0
                    total = 0.0
                    running_loss = 0
            training_iter +=1 
        torch.cuda.empty_cache()
        #evaluation
        acc = val(optimizer,snn,test_loader,test_dataset,batch_size,epoch)
        lr_scheduler.step()
        if not os.path.isdir(save_folder):
            os.mkdir(save_folder)
        if acc > bestacc:
            bestacc = acc
            print('===> Saving models...')


    torch.save(snn.state_dict(),'./'+save_folder+'/'+task+str(int(bestacc))+'.pkl')

use torch.cuda.amp.autocast for fusion prcision training
torch.Size([9000, 2, 42, 42, 8]) torch.Size([9000, 10])
torch.Size([1000, 2, 42, 42, 8]) torch.Size([1000, 10])
Total number of paramerters in networks is 11185379  
===> training models...


/home/lyh/anaconda3/envs/pytorch/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


=====> Epoch [0/100], Step [18/35], Loss: 38.38061 , Acc: 21.00694  
===> evaluating models...
val acc: 19.401041666666668 %  epoch: 0
===> Saving models...
===> training models...
=====> Epoch [1/100], Step [18/35], Loss: 34.58026 , Acc: 27.86458  
===> evaluating models...
val acc: 19.921875 %  epoch: 1
===> Saving models...
===> training models...
=====> Epoch [2/100], Step [18/35], Loss: 32.03172 , Acc: 34.63542  
===> evaluating models...
val acc: 34.505208333333336 %  epoch: 2
===> Saving models...
===> training models...
=====> Epoch [3/100], Step [18/35], Loss: 29.81617 , Acc: 40.60330  
===> evaluating models...
val acc: 37.369791666666664 %  epoch: 3
===> Saving models...
===> training models...
=====> Epoch [4/100], Step [18/35], Loss: 28.13913 , Acc: 44.53125  
===> evaluating models...
val acc: 37.239583333333336 %  epoch: 4
===> training models...
=====> Epoch [5/100], Step [18/35], Loss: 26.04518 , Acc: 48.41580  
===> evaluating models...
val acc: 33.463541666666664 %  

=====> Epoch [42/100], Step [18/35], Loss: 2.69218 , Acc: 94.57465  
===> evaluating models...
val acc: 46.223958333333336 %  epoch: 42
===> training models...
=====> Epoch [43/100], Step [18/35], Loss: 2.58852 , Acc: 95.33420  
===> evaluating models...
val acc: 46.484375 %  epoch: 43
===> training models...
=====> Epoch [44/100], Step [18/35], Loss: 2.50767 , Acc: 95.33420  
===> evaluating models...
val acc: 41.145833333333336 %  epoch: 44
===> training models...
=====> Epoch [45/100], Step [18/35], Loss: 2.26178 , Acc: 96.18056  
===> evaluating models...
val acc: 47.005208333333336 %  epoch: 45
===> training models...
=====> Epoch [46/100], Step [18/35], Loss: 2.44390 , Acc: 95.31250  
===> evaluating models...
val acc: 45.182291666666664 %  epoch: 46
===> training models...
=====> Epoch [47/100], Step [18/35], Loss: 1.96422 , Acc: 96.35416  
===> evaluating models...
val acc: 43.75 %  epoch: 47
===> training models...
=====> Epoch [48/100], Step [18/35], Loss: 2.16297 , Acc: 95.7

val acc: 47.395833333333336 %  epoch: 84
===> training models...
=====> Epoch [85/100], Step [18/35], Loss: 1.55838 , Acc: 97.15712  
===> evaluating models...
val acc: 47.916666666666664 %  epoch: 85
===> training models...
=====> Epoch [86/100], Step [18/35], Loss: 1.70104 , Acc: 96.85330  
===> evaluating models...
val acc: 47.65625 %  epoch: 86
===> training models...
=====> Epoch [87/100], Step [18/35], Loss: 2.02175 , Acc: 96.33247  
===> evaluating models...
val acc: 48.046875 %  epoch: 87
===> training models...
=====> Epoch [88/100], Step [18/35], Loss: 1.22211 , Acc: 97.74306  
===> evaluating models...
val acc: 49.088541666666664 %  epoch: 88
===> training models...
=====> Epoch [89/100], Step [18/35], Loss: 1.36613 , Acc: 97.30903  
===> evaluating models...
val acc: 47.786458333333336 %  epoch: 89
===> training models...
=====> Epoch [90/100], Step [18/35], Loss: 1.17295 , Acc: 98.06857  
===> evaluating models...
val acc: 52.213541666666664 %  epoch: 90
===> Saving models

===> evaluating models...
val acc: 41.015625 %  epoch: 25
===> training models...
=====> Epoch [26/100], Step [18/35], Loss: 4.69397 , Acc: 91.40625  
===> evaluating models...
val acc: 45.572916666666664 %  epoch: 26
===> Saving models...
===> training models...
=====> Epoch [27/100], Step [18/35], Loss: 5.04140 , Acc: 90.19097  
===> evaluating models...
val acc: 41.276041666666664 %  epoch: 27
===> training models...
=====> Epoch [28/100], Step [18/35], Loss: 4.41929 , Acc: 91.79688  
===> evaluating models...
val acc: 40.104166666666664 %  epoch: 28
===> training models...
=====> Epoch [29/100], Step [18/35], Loss: 3.93073 , Acc: 92.64323  
===> evaluating models...
val acc: 39.192708333333336 %  epoch: 29
===> training models...
=====> Epoch [30/100], Step [18/35], Loss: 3.55604 , Acc: 93.48959  
===> evaluating models...
val acc: 41.666666666666664 %  epoch: 30
===> training models...
=====> Epoch [31/100], Step [18/35], Loss: 3.54638 , Acc: 93.01215  
===> evaluating models...
v

=====> Epoch [68/100], Step [18/35], Loss: 1.77516 , Acc: 96.80990  
===> evaluating models...
val acc: 49.088541666666664 %  epoch: 68
===> training models...
=====> Epoch [69/100], Step [18/35], Loss: 1.92491 , Acc: 96.28906  
===> evaluating models...
val acc: 48.307291666666664 %  epoch: 69
===> training models...
=====> Epoch [70/100], Step [18/35], Loss: 1.71810 , Acc: 96.80990  
===> evaluating models...
val acc: 41.536458333333336 %  epoch: 70
===> training models...
=====> Epoch [71/100], Step [18/35], Loss: 1.73608 , Acc: 96.80990  
===> evaluating models...
val acc: 44.791666666666664 %  epoch: 71
===> training models...
=====> Epoch [72/100], Step [18/35], Loss: 1.55568 , Acc: 97.22222  
===> evaluating models...
val acc: 47.786458333333336 %  epoch: 72
===> training models...
=====> Epoch [73/100], Step [18/35], Loss: 1.64527 , Acc: 96.72309  
===> evaluating models...
val acc: 47.786458333333336 %  epoch: 73
===> training models...
=====> Epoch [74/100], Step [18/35], Los